# Pràctiques de Nous Usos de la Informàtica

**Nom de les persones del grup:** Xavi Cano & Orlando Manjarrez

# Pràctica 2. Recomanadors

La base de dades movielens 1M (http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·licules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online MovieLens (http://www.movielens.org/). 

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball. 

+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [1]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')


### Inspecció de les taules

In [2]:
users[:10]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [3]:
users[-10:]

,user_id,gender,age,occupation,zip
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [4]:
ratings[-10:]

,user_id,movie_id,rating,timestamp
1000199,6040,2022,5,956716207
1000200,6040,2028,5,956704519
1000201,6040,1080,4,957717322
1000202,6040,1089,4,956704996
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [5]:
ratings[:10]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [7]:
ratings.sort_values(by='movie_id')[:5]

,user_id,movie_id,rating,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317


In [8]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## 2.1 Exemple: Fent càlculs sobre DataFrames.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**. 

El primer pas a obtenir una única estructura que contingui tota la informació. 

Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció:

In [11]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificar d'usuari
data.sort_values(by='user_id')[:10]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
28501,1,48,5,978824351,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance
13819,1,938,4,978301752,F,1,10,48067,Gigi (1958),Musical
51327,1,1207,4,978300719,F,1,10,48067,To Kill a Mockingbird (1962),Drama
31152,1,1721,4,978300055,F,1,10,48067,Titanic (1997),Drama|Romance
37916,1,2762,4,978302091,F,1,10,48067,"Sixth Sense, The (1999)",Thriller
18472,1,2687,3,978824268,F,1,10,48067,Tarzan (1999),Animation|Children's
45685,1,2692,4,978301570,F,1,10,48067,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance
22832,1,720,3,978300760,F,1,10,48067,Wallace & Gromit: The Best of Aardman Animatio...,Animation
32771,1,745,3,978824268,F,1,10,48067,"Close Shave, A (1995)",Animation|Comedy|Thriller


La funció ``ix`` ens permet obtenir un subconjunt de files i/o columnes:

In [12]:
print data.ix[1]
print data.ix[3:4]

user_id                                            2
movie_id                                        1193
rating                                             5
timestamp                                  978298413
gender                                             M
age                                               56
occupation                                        16
zip                                            70072
title         One Flew Over the Cuckoo's Nest (1975)
genres                                         Drama
Name: 1, dtype: object
   user_id  movie_id  rating  timestamp gender  age  occupation    zip  \
3       15      1193       4  978199279      M   25           7  22903   
4       17      1193       5  978158471      M   50           1  95350   

                                    title genres  
3  One Flew Over the Cuckoo's Nest (1975)  Drama  
4  One Flew Over the Cuckoo's Nest (1975)  Drama  


Els índexs Boolans ens permeten seleccionar una part de la taula que compleix una condició.

In [13]:
# comptem quin tant per cent de ratings estan fets per una dona
print (data[data['gender']=='F']['rating'].count())/float(data['rating'].count()), '%'

0.246388504802 %


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes:

In [14]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

age,1,18,25,35,45,50,56
title,,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000,NaN
'Night Mother (1986),2.000000,4.666667,3.423077,2.904762,3.833333,3.555556,4.333333
'Til There Was You (1997),3.500000,2.500000,2.666667,2.900000,2.333333,2.500000,2.666667
"'burbs, The (1989)",4.500000,3.244444,2.652174,2.818182,2.545455,3.208333,2.666667
...And Justice for All (1979),3.000000,3.428571,3.724138,3.657143,4.100000,3.551724,3.928571
1-900 (1994),NaN,NaN,2.000000,NaN,NaN,NaN,3.000000
10 Things I Hate About You (1999),3.745455,3.415020,3.432950,3.102941,3.258065,3.629630,4.000000
101 Dalmatians (1961),3.514286,3.295082,3.613757,3.826087,3.976744,3.650000,3.190476
101 Dalmatians (1996),3.088235,2.467742,2.928571,3.279570,3.482759,3.400000,3.555556


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [15]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
1-900 (1994),2.000000,3.000000
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215


Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()`` el nombre.

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [16]:
ratings_by_title = data.groupby('title').size()
print ratings_by_title

title
$1,000,000 Duck (1971)                              37
'Night Mother (1986)                                70
'Til There Was You (1997)                           52
'burbs, The (1989)                                 303
...And Justice for All (1979)                      199
1-900 (1994)                                         2
10 Things I Hate About You (1999)                  700
101 Dalmatians (1961)                              565
101 Dalmatians (1996)                              364
12 Angry Men (1957)                                616
13th Warrior, The (1999)                           750
187 (1997)                                          55
2 Days in the Valley (1996)                        286
20 Dates (1998)                                    139
20,000 Leagues Under the Sea (1954)                575
200 Cigarettes (1999)                              181
2001: A Space Odyssey (1968)                      1716
2010 (1984)                                        470
24 7

Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [17]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``: 

In [18]:
mean_ratings = mean_ratings.ix[active_titles]
mean_ratings.sort_index()[:10]

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
"13th Warrior, The (1999)",3.112000,3.168000
2 Days in the Valley (1996),3.488889,3.244813
"20,000 Leagues Under the Sea (1954)",3.670103,3.709205
2001: A Space Odyssey (1968),3.825581,4.129738


Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [20]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [21]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre home si dones:

In [23]:
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:15]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones: 

In [24]:
sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions: 

In [26]:
# Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std()
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.ix[active_titles]
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

### EXERCICI 1

+ Donada la taula ``data``, calcula la puntuació mitjana de cada usuari. 

In [1]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')

data = pd.merge(pd.merge(ratings, users), movies)

#Puntuació mitjana de cada usuari
mean_by_user = data.groupby('user_id')['rating'].mean()
#Fem un print nomes de les 10 primeres ja que l'output es bastant llarg
print mean_by_user[:10]

user_id
1     4.188679
2     3.713178
3     3.901961
4     4.190476
5     3.146465
6     3.901408
7     4.322581
8     3.884892
9     3.735849
10    4.114713
Name: rating, dtype: float64


+ Quina és la pel·lícula més ben puntuada? (Guarda aquest valor en una variable ``string``)

In [2]:
#Mitjana de cada usuari agrupada per titol
mitja_per_usuari = data.groupby('title')['rating'].mean()
#Index mes alt puntuat dels titols
peli_max_puntuacio=mitja_per_usuari.idxmax()
print peli_max_puntuacio


Baby, The (1973)


### EXERCICI 2

+ Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada.<br> 


In [3]:
# la vostra solució aquí
def top_movie(dataFrame,usr):
    #Pivotar la taula para ordenarla por usuarios y obtener su mejor pelicula
    puntuacions=dataFrame.pivot_table('rating',index='user_id',columns='title')[usr-1:usr]
    #Retorno la pelicula que l'usuari a puntuat mes
    return puntuacions.max().idxmax() 

print top_movie(data,10)

Absent Minded Professor, The (1961)


# Construcció d'un recomanador.

El següent codi, donat el conjunt de dades, construeix un conjunt d'entrenament i un conjunt  de test:

In [4]:
import numpy as np

# generem subconjunts de training i test
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.ix[sampled_ids, 'for_testing'] = True
    return df

data['for_testing'] = False
grouped = data.groupby('user_id', group_keys=False).apply(assign_to_set)
movielens_train = data[grouped.for_testing == False]
movielens_test = data[grouped.for_testing == True]
print movielens_train.shape
print movielens_test.shape

(797758, 11)
(202451, 11)


La següent funció **evaluate**(estimate), donat un conjunt de dades d'entrenament i un conjunt de dades de test ens avalua la precisió d'un sistema de recomanació que li passem per paràmetre. Per a cadascun dels elements del conjunt de test haurem de pronosticar el seu valor i comparar-lo amb el valor real que l'usuari li ha asignat. La mesura que utilizarem per avaluar el sistema és la root-mean-square error (rsme)

In [5]:
# definim una funció per avaluar el resultat de la recomanació.

def compute_rmse(y_pred, y_true):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def evaluate(estimate,test=movielens_test):
    ids_to_estimate = zip(test['user_id'], test['movie_id'])
    estimated = np.array([estimate(u,i) for (u,i) in ids_to_estimate])
    real = test.rating.values
    return compute_rmse(estimated, real)

### EXERCICI 3

Fes l'avaluació d'un a política de recomanació naïf d'un ítem a un usuari basada en assignar a l'ítem la mitja de les recomanacions d'aquell  usuari.

In [6]:
# definim una funció per avaluar el resultat de la recomanació.

import numpy as np

# generem subconjunts de training i test
def assign_to_set(df):
    np.random.seed([42]) #Es fixa la semilla per no variar els valors
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.ix[sampled_ids, 'for_testing'] = True
    return df

data['for_testing'] = False
grouped = data.groupby('user_id', group_keys=False).apply(assign_to_set)
movielens_train = data[grouped.for_testing == False]
movielens_test = data[grouped.for_testing == True]
print movielens_train.shape
print movielens_test.shape

def compute_rmse(y_pred, y_true):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def evaluate(estimate,test=movielens_test):
    ids_to_estimate = zip(test['user_id'], test['movie_id'])
    estimated = np.array([estimate(u,i) for (u,i) in ids_to_estimate])
    real = test.rating.values
    return compute_rmse(estimated, real)

#Diccionario donde se guardan todas las puntuacions de peliculas de cada usuario y items
dicUser={} #key usuario y contiene sus peliculs y ratings de estas
dicPelis={}#key pelicula y contiene sus ratings de cada usuario

#Funcion para cargar previamente los diccionarios con la información necesaria, poca para que no haya mucha carga
#Se a comprovado que al cargar los diccionarios previamente se reduce el tiempo de computacion a la mitat,
#Muestra es los tiempos que se detallan
def carregarDic(DataFrame):
    users=DataFrame.user_id.unique() #Obtener todos los ids de los usuarios del data
    for us in users: #Es guarden el valors de rating i mocie_id de cada usuari per no fer més cercas al DataFrame
        dicUser[us]=DataFrame.ix[DataFrame['user_id']==us,['rating','movie_id']].sort(ascending=False,columns='rating') #Ordre Rating
    pelis=DataFrame.movie_id.unique() #Obtener todos los ids de la peliculas del data
    for film in pelis: #Se guardan los valores de Rating y movie_id de cada pelicula para segmentar la información en el dic
        dicPelis[film]=DataFrame.ix[DataFrame['movie_id']==film,['rating','user_id']].sort(ascending=False,columns='rating') #Ordre Rating

#Muy importante cargar antes para minimizar el tiempo, ya que sino se cargan antes el proceso de learn en users tarda 11 horas
# Mientras que si se carga previamente tarda poco menos de 5 horas, y asi con los otros procesos
carregarDic(movielens_train)#Preparar el diccionario para mejorar los tiempos de calculo por operacion

(797758, 11)
(202451, 11)


In [7]:
#meanUs={}
def rec1(user_id, item_id,train=movielens_train):
    #Put your recomender here
    #Usuando diccionarios de media
    #if(user_id not in meanUs):
        #meanUs[user_id] = train.ix[train['user_id'] == user_id,'rating'].mean()
    #Para evitar cargar en exceso la memoria se puede usar los ya cargados anteriormente que mejoran el tiempo notablemente
    #return meanUs[user_id]
    if user_id not in dicUser:
        dicUser[user_id]=train.ix[train['user_id']==user_id,['rating','movie_id']].sort_values(ascending=False,columns='rating')
        
    return dicUser[user_id].rating.mean() #al devolver el diccionario todas las peliculas del usuario la media se hace al momento

%time evaluate(rec1) #Para obtener el tiempo que tarda al hacer el calculo
print 'Error: %s' % evaluate(rec1)

Wall time: 8.43 s
Error: 1.03376167779
